In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, ConcatDataset

import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
import json

In [2]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.losses import DiceLoss
from segmentation_models_pytorch.utils.metrics import IoU, Fscore, Accuracy

In [3]:
import matplotlib.pyplot as plt

# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image, 'gray')
    plt.show()

In [4]:
import random

SEED = 0
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

print(np.random.rand(5), torch.randn(5))

[0.5488135  0.71518937 0.60276338 0.54488318 0.4236548 ] tensor([ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845])


# Settings

In [5]:
root = os.getcwd()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

BATCH = 4

GEN_IN_CHANNELS = 1
GEN_N_CLASSES = 1

DIS_IN_CHANNELS = 1
DIS_N_CLASSES = 2

E_IN_CHANNELS = 1
E_N_CLASSES = 2

EPOCH = 200

### Model Settings

#### encoder

In [6]:
ENCODER = "resnet152"
ENCODER_WEIGHT = None

#### decoder

In [7]:
DECODER_ATT = "scse"

#### head

In [8]:
GEN_ACT = "sigmoid"
DIS_ACT = "softmax"
E_ACT = "softmax"

# Epochs

In [9]:
from codes.losses import SSIMLoss
from codes.losses import MAELoss
from pytorch_msssim import ssim
from codes.metrics import PSNR, SNR, ContourEval
from codes.activation import Activation
from codes.utils import hu_clip_tensor
from codes.losses import PerceptualLoss
from kornia.filters.sobel import Sobel

In [10]:
def set_requires_grad(nets, requires_grad=False):
    """Set requies_grad=Fasle for all the networks to avoid unnecessary computations
    Parameters:
    nets (network list)   -- a list of networks
    requires_grad (bool)  -- whether the networks require gradients or not
    """
    if not isinstance(nets, list):
        nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad

In [11]:
def replace_relu_to_leakyReLU(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, nn.LeakyReLU())
        else:
            replace_relu_to_leakyReLU(child)

In [12]:
def replace_bn_to_instanceNorm(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.BatchNorm2d):
            bn = getattr(model, child_name)
            setattr(model, child_name, nn.InstanceNorm2d(bn.num_features))
        else:
            replace_bn_to_instanceNorm(child)

In [13]:
perceptual_ext = PerceptualLoss()

In [14]:
sobel_filter = Sobel().to(device)

In [15]:
@torch.no_grad()
def eval_epoch(generator, tf_discriminator, edge_discriminator, dataloader, device):
  
    # change mode to train and move to current device
    generator = generator.eval().to(device)
    tf_discriminator = tf_discriminator.eval().to(device)
    edge_discriminator = edge_discriminator.eval().to(device)
    
    ssim_ = []
    psnr_ = []
    mae_ = []
    snr_ = []
    
    air_ = []
    bone_ = []
    cont_ = []
 
    tf_acc = []
    edge_acc = []
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = generator(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        x = x[:, 0, :, :].unsqueeze(1)
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        total = x.size()[0]
       ############################
        # (1) Eval True/Fake D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################        
        y_cls = tf_discriminator(y)
        for _y_cls in y_cls:
            _y_cls = Activation(name=DIS_ACT)(_y_cls)
            _, _y_cls = torch.max(_y_cls.data, 1)
            correct = (_y_cls == torch.zeros(total, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]        
        
        y_pr_cls = tf_discriminator(y_pr)
        for _y_pr_cls in y_pr_cls:
            _y_pr_cls = Activation(name=DIS_ACT)(_y_pr_cls)
            _, _y_pr_cls = torch.max(_y_pr_cls.data, 1)
            correct = (_y_pr_cls == torch.ones(total, dtype=torch.long, device=device)).sum().item()
            tf_acc += [correct / total]
        
       ############################
        # (1.1) Eval Edge D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################  
        edge_y_cls = edge_discriminator(edge_y)
        for _edge_y_cls in edge_y_cls:
            _edge_y_cls = Activation(name=DIS_ACT)(_edge_y_cls)
            _, _edge_y_cls = torch.max(_edge_y_cls.data, 1)
            correct = (_edge_y_cls == torch.zeros(total, dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]          
        
        edge_pr_cls = edge_discriminator(edge_y_pr)
        for _edge_pr_cls in edge_pr_cls:
            _edge_pr_cls = Activation(name=DIS_ACT)(_edge_pr_cls)
            _, _edge_pr_cls = torch.max(_edge_pr_cls.data, 1)
            correct = (_edge_pr_cls == torch.ones(total, dtype=torch.long, device=device)).sum().item()
            edge_acc += [correct / total]        
        
        ############################
        # (2) Eval G network: maximize log(D(G(z)))
        ###########################
        
        # main
        _ssim = ssim(y, y_pr, data_range=1.0, size_average=True) # return (N,)
        _psnr = PSNR()(y_pr, y, 1.0)
        _snr = SNR()(y_pr, y)
        _mae = MAELoss()(y_pr.float(), y.float())

        ssim_ += [_ssim.item()]
        psnr_ += [_psnr.item()]
        mae_ += [_mae.item()]
        snr_ += [_snr.item()]
        
        # auxilary
        _min = VIEW_BOUND[0]
        _max = VIEW_BOUND[1]
        air_window =AIR_BOUND
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(air_pr, air_x)
        air_ += [dice.item()]
               
        bone_window = BONE_BOUND
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        dice = Fscore()(bone_pr, bone_x)
        bone_ += [dice.item()]

        for b in range(x.size()[0]):
            cont = ContourEval()(y_pr[b, :, :, :], x[b, :, :, :])
            cont_ += [cont.item()]

    return  sum(ssim_)/len(ssim_), sum(psnr_)/len(psnr_), sum(snr_)/len(snr_), sum(mae_)/len(mae_), \
                    sum(air_)/len(air_), sum(bone_)/len(bone_), sum(cont_)/len(cont_), sum(tf_acc)/len(tf_acc), sum(edge_acc)/len(edge_acc)

In [16]:
@torch.no_grad()
def test_epoch(iid, model, dataloader, device, save=False, path=None):
  
    # change mode to train and move to current device
    model = model.eval().to(device)
    
    for index, data in tqdm(enumerate(dataloader)):

        x, y, air_x, bone_x, *_ = data

        x = x.to(device)
        y = y.to(device)
        air_x = air_x.to(device)
        bone_x = bone_x.to(device)

        y_pr = model(x)
        y_pr = Activation(name=GEN_ACT)(y_pr) # zipped value to [0, 1]
        x = x[:, 0, :, :].unsqueeze(1)
        
        edge_x = sobel_filter(x)
        edge_y_pr = sobel_filter(y_pr)
        edge_y = sobel_filter(y)
        
        _min = VIEW_BOUND[0]
        _max = VIEW_BOUND[1]
        air_window =AIR_BOUND
        upper = ((air_window[1]) - (_min))/(_max-(_min))
        lower = ((air_window[0]) - (_min))/(_max-(_min))
        air_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)
        
        bone_window = BONE_BOUND
        upper = ((bone_window[1]) - (_min))/(_max-(_min))
        lower = ((bone_window[0]) - (_min))/(_max-(_min))
        bone_pr = hu_clip_tensor(y_pr.double(), (lower, upper), None, True)

        for b in range(x.shape[0]):           
            tmp_y = y[b, :, :, :].unsqueeze(0)
            tmp_y_pr = y_pr[b, :, :, :].unsqueeze(0)
            tmp_air = air_x[b, :, :, :].unsqueeze(0)
            tmp_air_pr = air_pr[b, :, :, :].unsqueeze(0)
            tmp_bone = bone_x[b, :, :, :].unsqueeze(0)
            tmp_bone_pr = bone_pr[b, :, :, :].unsqueeze(0)
            
            _ssim = ssim(tmp_y, tmp_y_pr, data_range=1.0, size_average=True) # return (N,)
            _psnr = PSNR()(tmp_y_pr, tmp_y, 1.0)
            _snr = SNR()(tmp_y_pr, tmp_y)
            _mae = MAELoss()(tmp_y_pr.float(), tmp_y.float())
            _air = Fscore()(tmp_air_pr, tmp_air)
            _bone = Fscore()(tmp_bone_pr, tmp_bone)
            _cont = ContourEval()(tmp_y_pr, x[b, :, :, :])
            
            scores = {
                "ssim score": _ssim.item(),
                "psnr score": _psnr.item(),
                "snr score": _snr.item(),
                "mae error": _mae.item(),
                "air dice score": _air.item(),
                "bone dice score": _bone.item(),
                "contour dice score": _cont.item()
            }
            
            __cbct = (x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct = (tmp_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_pred = (tmp_y_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __ct_masked = __ct_pred * tmp_air.squeeze().cpu().numpy()
            __air = (tmp_air.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __air_pr = (tmp_air_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone = (tmp_bone.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __bone_pr = (tmp_bone_pr.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_pr = (edge_y_pr[b, :].squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_x = (edge_x.squeeze().cpu().numpy() * 255).astype(np.uint8)
            __edge_y = (edge_y.squeeze().cpu().numpy() * 255).astype(np.uint8)
            
#             visualize(
#                 cbct = __cbct,
#                 ct_y = __ct,
#                 ct_pred = __ct_pred,
#                 edge_pred = __edge_pr,
#                 edge_x = __edge_x,
#                 edge_y = __edge_y,
#                 air = __air,
#                 air_pr = __air_pr,
#                 bone = __bone,
#                 bone_pr = __bone_pr
#             )
            
            if save:
                path_dir = os.path.join(path, "file_{}".format(iid))
                try:
                    os.mkdir(path_dir)
                except FileExistsError:
                    pass
                
                cv2.imwrite(os.path.join(path_dir, "cbct.jpg"), __cbct)
                cv2.imwrite(os.path.join(path_dir, "ct.jpg"), __ct)
                cv2.imwrite(os.path.join(path_dir, "ct_pred.jpg"), __ct_pred)
                cv2.imwrite(os.path.join(path_dir, "air.jpg"), __air)
                cv2.imwrite(os.path.join(path_dir, "air_pred.jpg"), __air_pr)
                cv2.imwrite(os.path.join(path_dir, "bone.jpg"), __bone)
                cv2.imwrite(os.path.join(path_dir, "bone_pred.jpg"), __bone_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_pred.jpg"), __edge_pr)
                cv2.imwrite(os.path.join(path_dir, "edge_cbct.jpg"), __edge_x)
                cv2.imwrite(os.path.join(path_dir, "edge_ct.jpg"), __edge_y)
                with open(os.path.join(path_dir, "scores.txt"), "w") as file:
                    file.write(json.dumps(scores))                
            iid += 1

# Discriminator

In [17]:
from codes.activation import Activation
import torchvision.models as models
from codes.losses import MultiScaleHeads

In [18]:
class Discriminator(nn.Module):
    def __init__(self, in_channel=1, n_classes=2, activation=None):
        
        super(Discriminator, self).__init__()
        
        encoder = smp.Unet(encoder_name="resnet18", in_channels=in_channel, classes=n_classes).encoder
        self.encoder = encoder
        self.fc = MultiScaleHeads(n_classes=n_classes, channels=(512, ), activation=None)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.fc([x[-1]])
        return x

# Generator

In [19]:
import segmentation_models_pytorch as smp
from codes.decoder import UnetDecoder
from segmentation_models_pytorch.base.heads import SegmentationHead

In [20]:
class Generator(nn.Module):
    def __init__(self, encoder_name, encoder_weights, in_channels, classes, attention_type):
        
        super(Generator, self).__init__()
        
        self.encoder = smp.Unet(encoder_name=encoder_name, encoder_weights=encoder_weights, in_channels=in_channels, classes=classes).encoder
        self.decoder = UnetDecoder(
                            encoder_channels=[in_channels, 0, 0, 0, 0, 2048],
                            decoder_channels=[512, 256, 128, 64, 16],
                            n_blocks=5,
                            use_batchnorm=True,
                            attention_type=attention_type)
        self.head =  SegmentationHead(
                        in_channels=16,
                        out_channels=classes,
                        activation=None)
        
        
    def forward(self, x):
        x = self.encoder(x)[-1]
        x = self.decoder(*(None, None, None, None, x))
        x = self.head(x)
        
        return x

# Read Data

In [21]:
import glob
from codes.dataset import Dataset, DicomDataset, DicomsDataset
import codes.augmentation as aug

In [22]:
# run_name = wandb.run.name
run_name = "dark-snowball-256"
ELECTRON = True
COORD = False

In [23]:
VIEW_BOUND = (-500, 500)
AIR_BOUND = (-500, -499)
BONE_BOUND = (255, 256)
if ELECTRON:
    VIEW_BOUND = (0.5, 1.5)
    AIR_BOUND = (0.5, 0.5009)
    BONE_BOUND = (1.2, 1.2009)  

In [24]:
checkpoint = torch.load(os.path.join("weight-gan", "{}.pth".format(run_name)))

In [25]:
model = smp.Unet(encoder_name=ENCODER, 
                  encoder_weights=ENCODER_WEIGHT, 
                  in_channels=GEN_IN_CHANNELS, 
                  classes=GEN_N_CLASSES,
                 decoder_attention_type=DECODER_ATT)
replace_relu_to_leakyReLU(model)
replace_bn_to_instanceNorm(model)
model.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [26]:
model

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (relu): LeakyReLU(negative_slope=0.01)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (relu): LeakyReLU(negative_slope=0.

In [27]:
tf_discriminator = Discriminator(in_channel=DIS_IN_CHANNELS, n_classes=DIS_N_CLASSES, activation=DIS_ACT)
replace_relu_to_leakyReLU(tf_discriminator)
replace_bn_to_instanceNorm(tf_discriminator)
tf_discriminator.load_state_dict(checkpoint["tf_discriminator"])

<All keys matched successfully>

In [28]:
edge_discriminator = Discriminator(in_channel=E_IN_CHANNELS, n_classes=E_N_CLASSES, activation=E_ACT)
replace_relu_to_leakyReLU(edge_discriminator)
replace_bn_to_instanceNorm(edge_discriminator)
edge_discriminator.load_state_dict(checkpoint["edge_discriminator"])

<All keys matched successfully>

# Pelvic

## Pelvic Test CBCT

In [29]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [30]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [31]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct"))
except FileExistsError:
    pass

In [32]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_cbct", patient_id)) 
    iid += len(scans)

28it [00:04,  6.63it/s]
27it [00:02, 10.50it/s]
26it [00:02, 10.77it/s]
28it [00:02,  9.57it/s]
27it [00:02, 10.38it/s]
28it [00:02, 10.30it/s]
27it [00:02, 10.62it/s]


In [33]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:21,  9.00it/s]


(0.8369941627167906,
 23.32993253737844,
 14.472364330791054,
 0.026961028020029292,
 0.9959209804475471,
 0.9254616880736084,
 0.32783887071135154,
 0.5,
 0.5)

## Pelvic Test CT

In [34]:
test_case_path = 'raw/test/*_*'
paths = sorted(glob.glob(test_case_path))

In [35]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [36]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct"))
except FileExistsError:
    pass

In [37]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "test_pelvic_ct", patient_id)) 
    iid += len(scans)

28it [00:02, 10.64it/s]
27it [00:02, 10.21it/s]
26it [00:02, 10.32it/s]
28it [00:02, 10.38it/s]
27it [00:02, 10.46it/s]
28it [00:02, 10.36it/s]
27it [00:02, 10.19it/s]


In [38]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

191it [00:19,  9.87it/s]


(0.9149061500714087,
 27.740805191519375,
 18.883236850119385,
 0.015414904651880576,
 0.995471479476626,
 0.9131370449856477,
 0.32578514633378436,
 0.5,
 0.5)

## Pelvic L1 CBCT

In [39]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [40]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [41]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct"))
except FileExistsError:
    pass

In [42]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_cbct", patient_id)) 
    iid += len(scans)

29it [00:02, 10.34it/s]
26it [00:02,  9.92it/s]
28it [00:02, 10.28it/s]
28it [00:02, 10.37it/s]
27it [00:02, 10.39it/s]
26it [00:02, 10.29it/s]
27it [00:02, 10.45it/s]
30it [00:02, 10.60it/s]
28it [00:02, 10.15it/s]
29it [00:02, 10.57it/s]
29it [00:02, 10.53it/s]
27it [00:02, 10.13it/s]
28it [00:02, 10.20it/s]
26it [00:02, 10.15it/s]
28it [00:02, 10.32it/s]
28it [00:02, 10.09it/s]
27it [00:02, 10.31it/s]
26it [00:02, 10.43it/s]
27it [00:02, 10.47it/s]
29it [00:02, 10.34it/s]
30it [00:02, 10.34it/s]
32it [00:03, 10.47it/s]
28it [00:02, 10.38it/s]
28it [00:02, 10.35it/s]
15it [00:01,  9.95it/s]
28it [00:02, 10.56it/s]
26it [00:02, 10.26it/s]
28it [00:02, 10.47it/s]
26it [00:02, 10.10it/s]
27it [00:02, 10.23it/s]
36it [00:03, 10.80it/s]
28it [00:02, 13.77it/s]
27it [00:03,  7.22it/s]
28it [00:02, 12.06it/s]
30it [00:02, 12.30it/s]
27it [00:02, 12.04it/s]
26it [00:02, 11.36it/s]
29it [00:02, 11.88it/s]
27it [00:02, 12.17it/s]
27it [00:02, 12.24it/s]
28it [00:02, 12.21it/s]
26it [00:02, 11.

In [43]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [02:05, 11.62it/s]


(0.8248593457447149,
 22.535562679549972,
 13.476751000323194,
 0.0306950687362404,
 0.9953419000301863,
 0.8889278699680979,
 0.34583704814435984,
 0.5020590253946465,
 0.49245024021962935)

## Pelvic L1 CT

In [44]:
test_case_path = 'L1_pelvic_processed/reg_pelvic_l1/*_*'
paths = sorted(glob.glob(test_case_path))

In [45]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [46]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct"))
except FileExistsError:
    pass

In [47]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L1_pelvic_ct", patient_id)) 
    iid += len(scans)

29it [00:02, 11.96it/s]
26it [00:02, 11.63it/s]
28it [00:02, 11.93it/s]
28it [00:02, 11.85it/s]
27it [00:02, 12.11it/s]
26it [00:02, 11.96it/s]
27it [00:02, 11.81it/s]
30it [00:02, 12.19it/s]
28it [00:02, 11.72it/s]
29it [00:02, 12.21it/s]
29it [00:02, 11.87it/s]
27it [00:02, 12.04it/s]
28it [00:02, 11.59it/s]
26it [00:02, 12.25it/s]
28it [00:02, 12.24it/s]
28it [00:02, 11.91it/s]
27it [00:02, 10.30it/s]
26it [00:03,  8.48it/s]
27it [00:02,  9.71it/s]
29it [00:02,  9.85it/s]
30it [00:03,  9.85it/s]
32it [00:03, 10.50it/s]
28it [00:02, 10.14it/s]
28it [00:02,  9.95it/s]
15it [00:01,  9.14it/s]
28it [00:02, 10.16it/s]
26it [00:02,  9.58it/s]
28it [00:03,  9.25it/s]
26it [00:02,  9.55it/s]
27it [00:02, 10.03it/s]
36it [00:03,  9.88it/s]
28it [00:02,  9.81it/s]
27it [00:02, 10.12it/s]
28it [00:02, 10.27it/s]
30it [00:02, 10.39it/s]
27it [00:02,  9.78it/s]
26it [00:02,  9.80it/s]
29it [00:02,  9.83it/s]
27it [00:02, 10.32it/s]
27it [00:02, 10.11it/s]
28it [00:02,  9.73it/s]
26it [00:02,  9.

In [48]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

1457it [02:28,  9.80it/s]


(0.9138746137641928,
 27.568046213750897,
 18.50923452855138,
 0.017370657388097878,
 0.9960760691291131,
 0.8977651660411382,
 0.370151856274957,
 0.49759780370624573,
 0.4921070693205216)

## Pelvic L2 CBCT

In [49]:
test_case_path = 'L2_pelvic_processed/reg_pelvic_l2/*_*'
paths = sorted(glob.glob(test_case_path))

In [50]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [51]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_cbct"))
except FileExistsError:
    pass

In [52]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="pelvic", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_cbct", patient_id)) 
    iid += len(scans)

26it [00:02,  9.96it/s]
26it [00:02,  9.74it/s]
26it [00:02, 10.07it/s]
27it [00:02,  9.94it/s]
31it [00:03, 10.10it/s]
27it [00:02, 10.22it/s]
27it [00:02, 10.52it/s]
26it [00:02, 12.58it/s]
28it [00:02, 10.18it/s]
28it [00:02, 11.66it/s]
27it [00:02, 12.07it/s]
27it [00:02, 11.37it/s]
26it [00:02, 12.20it/s]
26it [00:02, 11.90it/s]
26it [00:02, 11.31it/s]
26it [00:02, 11.60it/s]
26it [00:02, 11.48it/s]
26it [00:02, 12.09it/s]
27it [00:02, 11.56it/s]
26it [00:02, 11.92it/s]
26it [00:02, 12.51it/s]
28it [00:02, 11.74it/s]
28it [00:02, 12.12it/s]
29it [00:02, 11.55it/s]
28it [00:02, 11.62it/s]
26it [00:02, 11.62it/s]
27it [00:02, 11.68it/s]
26it [00:02, 12.05it/s]
28it [00:02, 11.90it/s]
30it [00:02, 11.71it/s]
27it [00:02, 11.20it/s]
29it [00:02, 11.71it/s]
27it [00:02, 12.15it/s]
27it [00:02, 11.21it/s]
26it [00:02, 11.66it/s]
26it [00:02, 11.04it/s]
26it [00:02, 11.90it/s]
26it [00:02, 12.38it/s]
27it [00:02, 11.33it/s]
26it [00:02, 11.87it/s]
26it [00:02, 11.32it/s]
26it [00:02, 11.

In [53]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="pelvic", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

3064it [04:54, 10.42it/s]


(0.8104987648697497,
 21.824304725731633,
 12.385347539816761,
 0.03350460440693691,
 0.9961022718596135,
 0.8391730130860474,
 0.40577712460930454,
 0.49820496083550914,
 0.49657310704960833)

## Pelvic L2 CT

In [54]:
test_case_path = 'L2_pelvic_processed/reg_pelvic_l2/*_*'
paths = sorted(glob.glob(test_case_path))

In [55]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [56]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct"))
except FileExistsError:
    pass

In [57]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i],
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="pelvic", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "L2_pelvic_ct", patient_id)) 
    iid += len(scans)

26it [00:02, 11.10it/s]
26it [00:02, 11.48it/s]
26it [00:02, 11.30it/s]
27it [00:02, 11.16it/s]
31it [00:02, 11.71it/s]
27it [00:02, 11.62it/s]
27it [00:02, 11.22it/s]
26it [00:02, 11.80it/s]
28it [00:02, 11.27it/s]
28it [00:02, 11.35it/s]
27it [00:02, 11.66it/s]
27it [00:02, 11.62it/s]
26it [00:02, 10.91it/s]
26it [00:02, 11.46it/s]
26it [00:02, 11.65it/s]
26it [00:02, 11.58it/s]
26it [00:02, 11.34it/s]
26it [00:02, 11.56it/s]
27it [00:02, 11.12it/s]
26it [00:02, 11.10it/s]
26it [00:02, 11.44it/s]
28it [00:02, 11.21it/s]
28it [00:02, 11.44it/s]
29it [00:02, 11.57it/s]
28it [00:02, 11.26it/s]
26it [00:02, 12.32it/s]
27it [00:02, 11.11it/s]
26it [00:02, 11.71it/s]
28it [00:02, 11.48it/s]
30it [00:02, 11.56it/s]
27it [00:02, 11.04it/s]
29it [00:02, 11.33it/s]
27it [00:02, 11.15it/s]
27it [00:02, 11.80it/s]
26it [00:02, 11.94it/s]
26it [00:02, 11.15it/s]
26it [00:02, 11.78it/s]
26it [00:02, 11.33it/s]
27it [00:02, 11.69it/s]
26it [00:02, 11.12it/s]
26it [00:02, 11.77it/s]
26it [00:02, 11.

In [58]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="pelvic", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

3064it [04:25, 11.54it/s]


(0.9199120519249931,
 27.93701118651009,
 18.4980539671746,
 0.01607658180065317,
 0.9957428364610488,
 0.8981766644917729,
 0.3758323189210361,
 0.49673629242819844,
 0.49673629242819844)

# Abdomen

In [59]:
test_case_path = 'abdomen/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Abdomen on CBCT

In [60]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [61]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct"))
except FileExistsError:
    pass

In [62]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="abdomen", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_cbct", patient_id)) 
    iid += len(scans)

28it [00:05,  5.19it/s]
78it [00:08,  9.43it/s]
50it [00:04, 10.37it/s]
29it [00:02, 10.00it/s]
50it [00:05,  9.19it/s]
71it [00:07,  8.94it/s]
39it [00:04,  9.53it/s]


In [63]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="abdomen", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

345it [00:36,  9.37it/s]


(0.7275603607080985,
 19.9215053309565,
 10.605391733197198,
 0.04734017518551453,
 0.9879020114362913,
 0.7188721927135895,
 0.28197589734758155,
 0.5449275362318841,
 0.5884057971014492)

## Abdomen on CT

In [64]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [65]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct"))
except FileExistsError:
    pass

In [66]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="abdomen", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "abdomen_test_ct", patient_id)) 
    iid += len(scans)

28it [00:03,  8.74it/s]
78it [00:08,  9.42it/s]
50it [00:05,  9.24it/s]
29it [00:02, 10.62it/s]
50it [00:04, 10.14it/s]
71it [00:06, 10.29it/s]
39it [00:03,  9.84it/s]


In [67]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="abdomen", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

345it [00:37,  9.18it/s]


(0.8793827927630881,
 25.772457343944605,
 16.456343728217526,
 0.022877019570897456,
 0.9887689910042589,
 0.813788909859676,
 0.3146283426038597,
 0.4855072463768116,
 0.5405797101449276)

# Chest

In [68]:
test_case_path = 'chest/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Chest on CBCT

In [69]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [70]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct"))
except FileExistsError:
    pass

In [71]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="chest", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_cbct", patient_id)) 
    iid += len(scans)

49it [00:04, 10.24it/s]
39it [00:03,  9.93it/s]
40it [00:03, 10.08it/s]
29it [00:03,  8.91it/s]
35it [00:03,  9.34it/s]
37it [00:03,  9.62it/s]
34it [00:03,  9.07it/s]


In [72]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="chest", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [00:27,  9.60it/s]


(0.759694053193009,
 20.21870634854973,
 9.921030876754354,
 0.04054451857767857,
 0.9825139110261581,
 0.7760820329294492,
 0.4351708801753621,
 0.49619771863117873,
 0.46387832699619774)

## Chest on CT

In [73]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [74]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct"))
except FileExistsError:
    pass

In [75]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="chest", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "chest_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "chest_test_ct", patient_id)) 
    iid += len(scans)

49it [00:04, 10.17it/s]
39it [00:03,  9.95it/s]
40it [00:03, 10.04it/s]
29it [00:03,  9.16it/s]
35it [00:03,  9.39it/s]
37it [00:03,  9.68it/s]
34it [00:03,  9.15it/s]


In [76]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="chest", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

263it [00:27,  9.58it/s]


(0.9106066748216578,
 27.12587577761806,
 16.828200420046034,
 0.01805247814029795,
 0.9819313109086795,
 0.8214622610173433,
 0.42176069361742,
 0.49619771863117873,
 0.34980988593155893)

# Headneck

In [77]:
test_case_path = 'headneck/test/*_*'
paths = sorted(glob.glob(test_case_path))

## Headneck on CBCT

In [78]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [79]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct"))
except FileExistsError:
    pass

In [80]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=False, electron=ELECTRON, position="headneck", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_cbct", patient_id)) 
    iid += len(scans)

58it [00:05, 10.71it/s]
55it [00:05, 10.94it/s]
55it [00:06,  9.12it/s]
56it [00:04, 12.36it/s]
56it [00:04, 12.37it/s]
55it [00:04, 12.39it/s]
56it [00:04, 12.46it/s]


In [81]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=False, electron=ELECTRON, position="headneck", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [00:33, 11.58it/s]


(0.8493035478360208,
 18.768044669304967,
 2.6708299199028698,
 0.042118561342168036,
 0.6957744708511491,
 0.694646830815039,
 0.042875286825671904,
 0.45907928388746805,
 0.47058823529411764)

## Headneck on CT

In [82]:
try:
    os.mkdir(os.path.join("eval-gan", run_name))
except FileExistsError:
    pass

In [83]:
try:
    os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct"))
except FileExistsError:
    pass

In [84]:
# read cbct and ct
iid = 0
for i in range(0, len(paths), 2):
    scans = DicomDataset(cbct_path=paths[i+1], ct_path=paths[i], 
                         geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                         identity=True, electron=ELECTRON, position="headneck", coord=COORD)
    patient_id = scans.patientID()
    testloader = torch.utils.data.DataLoader(scans, batch_size=1, shuffle=False, num_workers=4)
    try:
        os.mkdir(os.path.join("eval-gan", run_name, "hn_test_ct", patient_id))
    except FileExistsError:
        pass
    
    test_epoch(iid, model, testloader, device, True, os.path.join("eval-gan", run_name, "hn_test_ct", patient_id)) 
    iid += len(scans)

58it [00:04, 12.38it/s]
55it [00:04, 12.20it/s]
55it [00:04, 12.54it/s]
56it [00:04, 12.13it/s]
56it [00:04, 12.38it/s]
55it [00:04, 12.21it/s]
56it [00:04, 11.82it/s]


In [85]:
testset = DicomsDataset(test_case_path, geometry_aug=aug.get_validation_augmentation(), intensity_aug=None, 
                        identity=True, electron=ELECTRON, position="headneck", coord=COORD)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=4)
# score_ssim_test, score_psnr_test, score_snr_test, score_mae_test, \
#             score_air_test, score_bone_test, score_cont_test, score_tf_acc_test, score_edge_acc_test
eval_epoch(model, tf_discriminator, edge_discriminator, testloader, device)

391it [00:33, 11.53it/s]


(0.900774146132457,
 23.188640043253788,
 7.091425231045775,
 0.03245831024654381,
 0.6290922051905514,
 0.6061911021572767,
 0.10646141205129868,
 0.40281329923273657,
 0.4782608695652174)